# check logics for weighted calculation of loss

In [34]:
import torch
import torch.nn as nn

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, weighted="none", weight=2.0):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        if weighted=="none":
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.squeeze(), labels.squeeze())
        elif weighted=="ones_only":
            loss_fct = nn.CrossEntropyLoss(reduction="none")
            loss = loss_fct(logits.squeeze(), labels.squeeze())
            weights=torch.ones_like(loss)
            weights[labels[:,1]==1.0]=weight
            loss = torch.mean(loss * weights)
        return (loss, outputs) if return_outputs else loss

In [35]:
logits=torch.Tensor([[-2.345,4.930], [3.4902,1.293]]); logits.shape

torch.Size([2, 2])

In [36]:
labels=torch.Tensor([[0.2, 0.8], [0, 1.0]]); labels.shape

torch.Size([2, 2])

regular

In [5]:
loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(logits.squeeze(), labels.squeeze())

In [6]:
loss

tensor(1.8791)

weight by toxicity label -> deprecated

In [45]:
loss_fct = nn.CrossEntropyLoss(reduction="none")
loss = loss_fct(logits.squeeze(), labels.squeeze())
# loss = torch.sum(loss * labels[:,1])/len(logits)

In [46]:
loss

tensor([1.4557, 2.3026])

In [47]:
labels[:,1]

tensor([0.8000, 1.0000])

In [49]:
torch.sum(loss * labels[:,1])/torch.sum(labels[:,1])

tensor(1.9262)

In [51]:
# wrong!
# torch.mean(loss * labels[:,1]) # tensor(1.7336)

tensor(1.7336)

weight toxicity=1.0 samples

In [37]:
weight=2.0
loss_fct = nn.CrossEntropyLoss(reduction="none")
loss = loss_fct(logits.squeeze(), labels.squeeze())
# weights=torch.ones_like(loss)
# weights[labels[:,1]==1.0]=weight
# loss = torch.mean(loss * weights)

In [38]:
loss

tensor([1.4557, 2.3026])

In [39]:
weights=torch.ones_like(loss); weights

tensor([1., 1.])

In [40]:
weights[labels[:,1]==1.0]=weight; weights

tensor([1., 2.])

In [41]:
loss * weights

tensor([1.4557, 4.6051])

In [43]:
torch.sum(loss*weights)/torch.sum(weights)

tensor(2.0203)

In [44]:
# wrong!
#torch.mean(loss*weights) # tensor(3.0304)

tensor(3.0304)